[View in Colaboratory](https://colab.research.google.com/github/dkanunnikov/netology/blob/master/dk_cv4.ipynb)

# Yelp Restaurant Photo Classification

Описание задания:
Предсказание тегов ресторанов по фотографиям
Обучить модель предсказания тегов ресторанов по набору фото. Исходные данные и валидация на Kaggle в рамках контеста Yelp Restaurant Photo Classification. Шаблон ноутбука для подготовки решения: yelp-hw.ipynb .
Решение необходимо прислать в виде ссылки на ipython-ноутбука с указанием значения метрики на Leaderboard. Задание засчитывается при значение метрики на Leaderboard больше 0.7.

https://www.kaggle.com/c/yelp-restaurant-photo-classification

In [1]:
!pip install keras --upgrade
!pip install keras_applications --upgrade
!pip install kaggle


    100% |████████████████████████████████| 307kB 6.2MB/s 
    100% |████████████████████████████████| 51kB 10.3MB/s 
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6
Requirement already up-to-date: keras_applications in /usr/local/lib/python3.6/dist-packages (1.0.4)
    100% |████████████████████████████████| 51kB 2.5MB/s 
    100% |████████████████████████████████| 51kB 5.9MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/40/3a/1a/030a08ebef5e1aef1ee6f0fd2b70666a9363f48d4bb6f7c4b4
Successfully built kaggle


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.contrib import keras
from keras.models import Model, Sequential
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Concatenate
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras import backend as K


preprocess_input = keras.applications.vgg16.preprocess_input
VGG16 = keras.applications.VGG16

IMG_SIZE = (224, 224)

Using TensorFlow backend.


In [0]:
!pip install kaggle-cli > /dev/null
!rm -rf train_photos && rm -rf test_photos > /dev/null && rm -rf train > /dev/null && rm -rf test > /dev/null


In [4]:
!kaggle  config  set  -n  path  -v  /content

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 93, in authenticate
    permissions = os.stat(self.config).st_mode
FileNotFoundError: [Errno 2] No such file or directory: '/content/.kaggle/kaggle.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 7, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 122, in authenticate
    self.config_file + ' in the folder ' + self.config_dir)
ValueError: Unauthorized: you must download an API key from https://www.kaggle.com/<username>/account
Then put kaggle.json in the folder /content/.kaggle


In [5]:
!kg  download -u  -p   -c yelp-restaurant-photo-classification -f train_photos.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/train_photos.tgz

train_photos.tgz   5% |#                            | ETA:  0:03:20  31.5 MiB/s

train_photos.tgz 100% |#############################| Time: 0:03:38  30.6 MiB/s



In [6]:
!kg  download -u webotdel@mail.ru -p 64136413L  -c yelp-restaurant-photo-classification -f test_photos.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/test_photos.tgz

test_photos.tgz   5% |#                             | ETA:  0:03:44  28.5 MiB/s

test_photos.tgz 100% |##############################| Time: 0:03:49  29.5 MiB/s



In [7]:
!kg  download -u webotdel@mail.ru -p 64136413L  -c yelp-restaurant-photo-classification -f train.csv.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/train.csv.tgz

train.csv.tgz 100% |################################| Time: 0:00:00  13.8 KiB/s



In [8]:
!kg  download -u webotdel@mail.ru -p 64136413L  -c yelp-restaurant-photo-classification -f train_photo_to_biz_ids.csv.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/train_photo_to_biz_ids.csv.tgz

train_photo_to_biz_ids.csv.tgz 100% |###############| Time: 0:00:00   3.9 MiB/s



In [9]:
!kg  download -u webotdel@mail.ru -p 64136413L  -c yelp-restaurant-photo-classification -f test_photo_to_biz.csv.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/test_photo_to_biz.csv.tgz

test_photo_to_biz.csv.tgz 100% |####################| Time: 0:00:00  11.8 MiB/s



In [0]:
!tar -zxvf train_photos.tgz > /dev/null && tar -zxvf test_photos.tgz > /dev/null

In [0]:
!tar -zxvf train.csv.tgz > /dev/null

In [0]:
!tar -zxvf train_photo_to_biz_ids.csv.tgz > /dev/null

In [0]:
!tar -zxvf test_photo_to_biz.csv.tgz > /dev/null

In [14]:
!ls  -l

total 13848980
drwxr-xr-x 3 root root        4096 Aug  2 18:43 datalab
drwxr-xr-x 2  501 staff   14209024 Oct 30  2015 test_photos
-rw-r--r-- 1 root root  7102584231 Aug  4 15:38 test_photos.tgz
-rw-r--r-- 1  504 staff   15193591 Dec 17  2015 test_photo_to_biz.csv
-rw-r--r-- 1 root root     5019061 Aug  4 15:38 test_photo_to_biz.csv.tgz
-rw-r--r-- 1  501 staff      28760 Dec 15  2015 train.csv
-rw-r--r-- 1 root root        7287 Aug  4 15:38 train.csv.tgz
drwxr-xr-x 2  501 staff   13692928 Oct 30  2015 train_photos
-rw-r--r-- 1 root root  7026724672 Aug  4 15:34 train_photos.tgz
-rw-r----- 1  501 staff    2698404 Dec 14  2015 train_photo_to_biz_ids.csv
-rw-r--r-- 1 root root     1171455 Aug  4 15:38 train_photo_to_biz_ids.csv.tgz


## Загружаем разметку

In [0]:
train_biz_df = pd.read_csv('train.csv')
train_photos_df = pd.read_csv('train_photo_to_biz_ids.csv')
data_df = train_photos_df.merge(train_biz_df)

data_df.dropna(inplace=True)

test_photos_df = pd.read_csv('test_photo_to_biz.csv')



In [0]:
def get_image(path, img_id, img_size=(224, 224)):
    img = cv2.imread(os.path.join(path, '%s.jpg' % img_id))[:,:,::-1]
    img = preprocess_input(img.astype(np.float32))
    return cv2.resize(img, img_size)

def to_dense(labels):
    result = [0] * 9
    for i in labels:
        result[i] = 1.
    return result

def train_generator(df, img_size=(224, 224), batch_size=32):
    while True:
        df = df.sample(frac=1).reset_index(drop=True)
        for i in range(0, len(df) // batch_size * batch_size, batch_size):
            X, y = [], []
            for _, row in df[i:i + batch_size].iterrows():
                X.append(get_image('train_photos', row['photo_id'], img_size))
                y.append(to_dense(map(int, str(row['labels']).split())))
            y = np.array(y)
            yield np.array(X), [y[:, i] for i in range(9)]

In [0]:
# TODO: build CNN model
# TODO: train CNN model using train_generator

In [0]:

from keras.callbacks import ModelCheckpoint
checkout = ModelCheckpoint('yelp_weights.h5', save_best_only=True, verbose=1)

In [19]:
#base_model = InceptionV3(
#    include_top = False,
#    weights = 'imagenet',
#    input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3),
#    pooling = 'avg'
#)

base_model = Xception(include_top = False,
                   weights = 'imagenet',
                   input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3),
                   pooling = 'avg'
           )


83689472/83683744 [==============================] - 16s 0us/step


In [0]:
# фиксируем все веса предобученной сети
for layer in base_model.layers:
    layer.trainable = False

In [21]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [0]:
x = base_model.output
predictions = []
for i in range(9):
    x0 = Dense(128, activation='relu')(x)
    x0 = Dropout(0.5)(x0)
    x0 = Dense(1, activation='sigmoid')(x0)
    predictions.append(x0)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data_df, test_size=0.01, random_state=123)
validation_data = next(train_generator(test_df, batch_size = len(test_df)))

In [24]:
!rm yelp_weights.h5

rm: cannot remove 'yelp_weights.h5': No such file or directory


In [25]:
model.fit_generator(
       train_generator(train_df),
        steps_per_epoch=10,
        epochs=5,
        validation_data=validation_data,
        callbacks=[checkout]
)

Epoch 1/5
10/10 [==============================] - 39s 4s/step - loss: 6.0522 - dense_2_loss: 0.5875 - dense_4_loss: 0.7013 - dense_6_loss: 0.6806 - dense_8_loss: 0.7373 - dense_10_loss: 0.7013 - dense_12_loss: 0.6379 - dense_14_loss: 0.6225 - dense_16_loss: 0.6688 - dense_18_loss: 0.7150 - dense_2_acc: 0.7781 - dense_4_acc: 0.5812 - dense_6_acc: 0.6219 - dense_8_acc: 0.4813 - dense_10_acc: 0.5094 - dense_12_acc: 0.6969 - dense_14_acc: 0.7312 - dense_16_acc: 0.5969 - dense_18_acc: 0.5312 - val_loss: 20.2666 - val_dense_2_loss: 1.6196 - val_dense_4_loss: 1.2764 - val_dense_6_loss: 1.2319 - val_dense_8_loss: 2.6462 - val_dense_10_loss: 3.6266 - val_dense_12_loss: 1.5399 - val_dense_14_loss: 2.0801 - val_dense_16_loss: 2.8743 - val_dense_18_loss: 3.3716 - val_dense_2_acc: 0.7783 - val_dense_4_acc: 0.4685 - val_dense_6_acc: 0.5243 - val_dense_8_acc: 0.5332 - val_dense_10_acc: 0.5835 - val_dense_12_acc: 0.7434 - val_dense_14_acc: 0.7711 - val_dense_16_acc: 0.6347 - val_dense_18_acc: 0.5060



Epoch 00004: val_loss did not improve from 16.03878
Epoch 5/5
10/10 [==============================] - 31s 3s/step - loss: 5.3856 - dense_2_loss: 0.5330 - dense_4_loss: 0.6530 - dense_6_loss: 0.5871 - dense_8_loss: 0.7150 - dense_10_loss: 0.6403 - dense_12_loss: 0.5211 - dense_14_loss: 0.4553 - dense_16_loss: 0.6244 - dense_18_loss: 0.6562 - dense_2_acc: 0.7656 - dense_4_acc: 0.6312 - dense_6_acc: 0.6562 - dense_8_acc: 0.4656 - dense_10_acc: 0.6625 - dense_12_acc: 0.7656 - dense_14_acc: 0.7906 - dense_16_acc: 0.6406 - dense_18_acc: 0.6562 - val_loss: 18.8599 - val_dense_2_loss: 2.2994 - val_dense_4_loss: 1.9598 - val_dense_6_loss: 1.4634 - val_dense_8_loss: 2.0943 - val_dense_10_loss: 2.8375 - val_dense_12_loss: 2.2275 - val_dense_14_loss: 2.5860 - val_dense_16_loss: 1.7121 - val_dense_18_loss: 1.6799 - val_dense_2_acc: 0.7783 - val_dense_4_acc: 0.4020 - val_dense_6_acc: 0.5844 - val_dense_8_acc: 0.5350 - val_dense_10_acc: 0.5720 - val_dense_12_acc: 0.7438 - val_dense_14_acc: 0.7732 -

In [26]:
!ls -l


total 13958640
drwxr-xr-x 3 root root        4096 Aug  2 18:43 datalab
drwxr-xr-x 2  501 staff   14209024 Oct 30  2015 test_photos
-rw-r--r-- 1 root root  7102584231 Aug  4 15:38 test_photos.tgz
-rw-r--r-- 1  504 staff   15193591 Dec 17  2015 test_photo_to_biz.csv
-rw-r--r-- 1 root root     5019061 Aug  4 15:38 test_photo_to_biz.csv.tgz
-rw-r--r-- 1  501 staff      28760 Dec 15  2015 train.csv
-rw-r--r-- 1 root root        7287 Aug  4 15:38 train.csv.tgz
drwxr-xr-x 2  501 staff   13692928 Oct 30  2015 train_photos
-rw-r--r-- 1 root root  7026724672 Aug  4 15:34 train_photos.tgz
-rw-r----- 1  501 staff    2698404 Dec 14  2015 train_photo_to_biz_ids.csv
-rw-r--r-- 1 root root     1171455 Aug  4 15:38 train_photo_to_biz_ids.csv.tgz
-rw-r--r-- 1 root root   112291600 Aug  4 15:45 yelp_weights.h5


## Получаем предсказания

In [0]:
model.load_weights('yelp_weights.h5')

In [28]:
len(test_photos_df)

1190225

In [0]:
class TestSequence(keras.utils.Sequence):
    """ Класс для чтения батча """
    def __init__(self, df, batch_size):
        self._df = df
        self._batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self._df) / float(self._batch_size)))

    def __getitem__(self, idx):
        result = []
        sample = self._df[idx * self._batch_size:(idx + 1) * self._batch_size]
        for _, row in sample.iterrows():
            result.append(get_image('test_photos', row['photo_id']))
        return np.array(result)

In [30]:
test_photos_df.head()

,photo_id,business_id
0,317818,003sg
1,30679,003sg
2,455084,003sg
3,371381,003sg
4,86224,003sg


In [31]:
# читаем данные параллельно в 4потока и применяем модель
preds = model.predict_generator(TestSequence(test_photos_df, batch_size=100), workers=4, verbose=1)

   

ValueError: ignored

In [0]:
test_photos_df['labels'] = [[float(preds[j][i]) for j in range(9)] 
                            for i in range(len(test_photos_df))]

In [0]:
test_photos_df.head()

,photo_id,business_id
0,317818,003sg
1,30679,003sg
2,455084,003sg
3,371381,003sg
4,86224,003sg


In [0]:
business_df = test_photos_df.groupby('business_id')

In [0]:
submission = []
for k, labels in business_df['labels'].apply(list).iteritems():
    labels = np.array(labels).mean(axis=0)
    labels = np.where(labels > 0.5)[0]
    submission.append((k, ' '.join(map(str, labels))))

KeyError: ignored

In [0]:
submission_df = pd.DataFrame(submission, columns=('business_id', 'labels'))
submission_df.to_csv('submission.csv', index=False)
# 0.70970

In [0]:
files.download('submission.csv')